In [ ]:
%%pyspark

spark.sql("create database iowa")

In [3]:
%%pyspark

df = spark.read.load('abfss://iowabyyear@eightfive.dfs.core.windows.net/IowaLiquorSalesdf2022.parquet', format='parquet')
df.write.mode("overwrite").saveAsTable("iowa.Iowa2022")
iowa2022 = sqlContext.table("iowa.iowa2022")

StatementMeta(efsynapsespark, 20, 2, Finished, Available)

In [4]:
%%pyspark
from pyspark.sql.types import StringType, DecimalType, IntegerType

iowaPopulation = spark.read.load('abfss://iowafiles@eightfive.dfs.core.windows.net/IowaPopulation.csv', format='csv', header=True)


StatementMeta(efsynapsespark, 20, 3, Finished, Available)

In [6]:
from pyspark.sql.functions import regexp_replace, col, substring, length, split, lit
from pyspark.sql.types import IntegerType

iowaPopulationRefined = (
    iowaPopulation
        .withColumn("County", 
            split(col("County").substr(lit(2), length(col("County"))), " County").getItem(0))
         .withColumn("Population", 
            regexp_replace(col("Population"), ",", "")
            .astype(IntegerType()), 
        )
)
iowaPopulationRefined.write.mode("overwrite").saveAsTable("iowa.iowapopulation")

StatementMeta(efsynapsespark, 20, 5, Finished, Available)

In [11]:
display(iowa2022.limit(5))
iowa2022.printSchema()

StatementMeta(efsynapsespark, 20, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9750cae3-078c-422d-bfbe-5b81dfeac4f1)


root
 |-- InvoiceItemNumber: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- StoreNumber: long (nullable = true)
 |-- StoreName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZipCode: long (nullable = true)
 |-- StoreLocation: string (nullable = true)
 |-- CountyNumber: long (nullable = true)
 |-- County: string (nullable = true)
 |-- CategoryNumber: long (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- VendorNumber: long (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- ItemNumber: long (nullable = true)
 |-- ItemDescription: string (nullable = true)
 |-- Pack: long (nullable = true)
 |-- BottleVolumeMl: long (nullable = true)
 |-- StateBottleCost: double (nullable = true)
 |-- StateBottleRetail: double (nullable = true)
 |-- BottlesSold: long (nullable = true)
 |-- SaleDollars: double (nullable = true)
 |-- VolumeSoldLiters: double (nullable = true)
 |-- VolumeSoldGallons

In [12]:
display(iowaPopulationRefined.limit(5))
iowaPopulationRefined.printSchema()

StatementMeta(efsynapsespark, 20, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, c4a0dbcb-e47b-43a1-af4d-fc1b1918e060)


root
 |-- County: string (nullable = true)
 |-- Population: integer (nullable = true)

In [13]:
from pyspark.sql.functions import lower, sum, avg, col, initcap, round, date_format, trunc, first, row_number
from pyspark.sql.window import Window

rowNumberWindow = Window.partitionBy(col("Date")).orderBy(col("County"))
sumWindow1 = Window.partitionBy(col("Date"))

iowa2022agg = (
    iowa2022
        .withColumn("County", initcap("County"))
        .withColumn("Date", trunc(col("Date"), "month"))
        .filter( (col("Date").isNotNull()) & (col("Date") >= "2022-01-01") )
        .join(iowaPopulation, on="County", how="leftouter")
        .groupBy(col("Date"), col("County"))
        .agg(
            round(sum("SaleDollars")).alias("Sum of Sales"), 
            round(avg("SaleDollars"), 2).alias("Avg of Sales"), 
            first(iowaPopulation["Population"]).alias("Population"))
        .filter( (col("Sum of Sales").between(10, 1000000)) & (col("County").like("M%")) )
        .withColumn("Rank", row_number().over(rowNumberWindow))
        .withColumn("Sum per Month", sum(col("Sum of Sales")).over(sumWindow1))
        .orderBy(col("Date").desc(), col("County").asc())
        .show(20)
)

StatementMeta(efsynapsespark, 20, 12, Finished, Available)

+----------+----------+------------+------------+----------+----+-------------+
|      Date|    County|Sum of Sales|Avg of Sales|Population|Rank|Sum per Month|
+----------+----------+------------+------------+----------+----+-------------+
|2022-02-01|   Madison|     88990.0|      115.12|      null|   1|    1444671.0|
|2022-02-01|   Mahaska|    130904.0|      130.25|      null|   2|    1444671.0|
|2022-02-01|    Marion|    216441.0|      126.87|      null|   3|    1444671.0|
|2022-02-01|  Marshall|    342940.0|      151.07|      null|   4|    1444671.0|
|2022-02-01|     Mills|     64835.0|       88.09|      null|   5|    1444671.0|
|2022-02-01|  Mitchell|     37954.0|       90.15|      null|   6|    1444671.0|
|2022-02-01|    Monona|     46503.0|       56.44|      null|   7|    1444671.0|
|2022-02-01|    Monroe|     35228.0|       152.5|      null|   8|    1444671.0|
|2022-02-01|Montgomery|     78111.0|       166.9|      null|   9|    1444671.0|
|2022-02-01| Muscatine|    402765.0|    

In [14]:
%%sql

SELECT
   TRUNC(i22.Date, "MM") AS `Date`
  ,INITCAP(i22.County) AS `County`
  ,ROUND(SUM(i22.SaleDollars)) AS `Sum of Sales`
  ,ROUND(AVG(i22.SaleDollars), 2) AS `Avg of Sales`
  ,FIRST(ip.Population) AS `Population`
  ,ROW_NUMBER() OVER(PARTITION BY TRUNC(i22.Date, "MM") ORDER BY INITCAP(i22.County)) AS `Rank`
  ,SUM(SUM(ROUND(i22.SaleDollars))) OVER(PARTITION BY TRUNC(i22.Date, "MM")) AS `Sum per Month`
FROM iowa.iowa2022 i22
LEFT JOIN iowa.iowapopulation ip
    ON INITCAP(i22.County) = ip.County
WHERE `Date` IS NOT NULL AND `Date` >= "2022-01-01"
GROUP BY TRUNC(i22.Date, "MM"), INITCAP(i22.County)
HAVING `Sum of Sales` BETWEEN 10 AND 1000000 AND `County` LIKE "M%"
ORDER BY `Date` DESC, `County` ASC


StatementMeta(efsynapsespark, 20, 13, Finished, Available)

<Spark SQL result set with 20 rows and 7 fields>